## Monolingual Named Entity Recognition (Few Nerd Dataset) (Fine-Tuning)

Dataset Source: https://huggingface.co/datasets/DFKI-SLT/few-nerd

#### Install Necessary Libraries

In [1]:
%pip install transformers datasets evaluate seqeval
%pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=fdf44d0112ebc987734efe85827b2826d4c054e17d8409bf0b64f1875ec36345
  St

#### HuggingFace Access

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


#### Import Necessary Libraries

In [3]:
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import datasets
from datasets import load_dataset, DatasetDict
from datasets import Dataset, concatenate_datasets

import torch
from torch.nn.functional import cross_entropy

import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import Trainer, TrainingArguments

import evaluate

!git lfs install

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


#### Display Library Versions

In [4]:
print("Python :".rjust(18), sys.version[0:6])
print("NumPy :".rjust(18), np.__version__)
print("Pandas :".rjust(18), pd.__version__)
print("Torch :".rjust(18), torch.__version__)
print("Transformers :".rjust(18), transformers.__version__)
print("Datasets :".rjust(18), datasets.__version__)
print("Evaluate :".rjust(18), evaluate.__version__)
print("Sklearn :".rjust(18), sklearn.__version__)

          Python : 3.10.1
           NumPy : 1.22.4
          Pandas : 1.5.3
           Torch : 2.0.1+cu118
    Transformers : 4.30.2
        Datasets : 2.13.1
        Evaluate : 0.4.0
         Sklearn : 1.2.2


#### Ingest English Subset of Dataset

In [5]:
subsets_to_include = ["inter",
                      "intra",
                      "supervised",
                      ]

temp_data: []
train_data: Dataset
test_data: Dataset
eval_data: Dataset

for idx, s in enumerate(subsets_to_include):
    temp_data = load_dataset("DFKI-SLT/few-nerd", s)
    if idx != 0:
        train_data = concatenate_datasets([temp_data['train'], train_data])
        test_data = concatenate_datasets([temp_data['test'], test_data])
        eval_data = concatenate_datasets([temp_data['validation'], eval_data])
    else:
        train_data = temp_data['train']
        test_data = temp_data['test']
        eval_data = temp_data['validation']

print(f"Training Dataset Information: \n{train_data}")
print(f"Testing Dataset Information: \n{test_data}")
print(f"Evaluation Dataset Information: \n{eval_data}")

Generating train split: 0 examples [00:00, ? examples/s]

  0%|          | 0/3586051 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

  0%|          | 0/444827 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

Generating test split: 0 examples [00:00, ? examples/s]

  0%|          | 0/958765 [00:00<?, ?it/s]

Dataset few-nerd downloaded and prepared to /root/.cache/huggingface/datasets/DFKI-SLT___few-nerd/supervised/0.0.0/c4e2bba21055a1e5f9d70b1ede49778c6ef267f3a6240d54f12e43f72fce5dd4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Training Dataset Information: 
Dataset({
    features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
    num_rows: 361398
})
Testing Dataset Information: 
Dataset({
    features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
    num_rows: 95714
})
Evaluation Dataset Information: 
Dataset({
    features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
    num_rows: 56999
})


#### Split Dataset into DatasetDict

In [6]:
# Pack all of them into DatasetDict

ds = DatasetDict({
    'train': train_data,
    'test': test_data,
    'eval': eval_data})

print('Training data shape:', ds['train'].shape)
print('Testing data shape:', ds['test'].shape)
print('Validation data shape:', ds['eval'].shape)

Training data shape: (361398, 4)
Testing data shape: (95714, 4)
Validation data shape: (56999, 4)


#### Example

In [7]:
example = ds['train'][12]

example

{'id': '12',
 'tokens': ['Imelda',
  'de',
  "'",
  'Lambertazzi',
  'is',
  'a',
  '``',
  'melodramma',
  'tragico',
  "''",
  'or',
  'tragic',
  'opera',
  'in',
  'two',
  'acts',
  'by',
  'Gaetano',
  'Donizett',
  'i',
  'from',
  'a',
  'libretto',
  'by',
  'Andrea',
  'Leone',
  'Tottola',
  ',',
  'based',
  'on',
  'the',
  'tragedy',
  '``',
  'Imelda',
  "''",
  'by',
  'Gabriele',
  'Sperduti',
  '.'],
 'ner_tags': [1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  7,
  0,
  0,
  0,
  0,
  0,
  7,
  7,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  0],
 'fine_ner_tags': [6,
  6,
  6,
  6,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  51,
  51,
  0,
  0,
  0,
  0,
  0,
  51,
  51,
  51,
  0,
  0,
  0,
  0,
  0,
  0,
  6,
  6,
  6,
  6,
  6,
  0]}

#### Display Feature Information About Each Feature

In [8]:
for k, v in ds["train"].features.items():
    print(f"{k}: \n{v}\n")

id: 
Value(dtype='string', id=None)

tokens: 
Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

ner_tags: 
Sequence(feature=ClassLabel(names=['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product'], id=None), length=-1, id=None)

fine_ner_tags: 
Sequence(feature=ClassLabel(names=['O', 'art-broadcastprogram', 'art-film', 'art-music', 'art-other', 'art-painting', 'art-writtenart', 'building-airport', 'building-hospital', 'building-hotel', 'building-library', 'building-other', 'building-restaurant', 'building-sportsfacility', 'building-theater', 'event-attack/battle/war/militaryconflict', 'event-disaster', 'event-election', 'event-other', 'event-protest', 'event-sportsevent', 'location-GPE', 'location-bodiesofwater', 'location-island', 'location-mountain', 'location-other', 'location-park', 'location-road/railway/highway/transit', 'organization-company', 'organization-education', 'organization-government/governmentagency', 'organizati

#### Define Tag Values & Conversions Between String & Integer Values

In [9]:
label2id ={
    "O": 0,
    "art": 1,
    "building": 2,
    "event": 3,
    "location": 4,
    "organization": 5,
    "other": 6,
    "person": 7,
    "product": 8
    }

tag_values = list(label2id.keys())
NUM_OF_LABELS = len(tag_values)

id2label = {tag: idx for idx, tag in label2id.items()}

print(f"List of tag values: \n{tag_values}")
print(f"Number of NER Tags: \n{NUM_OF_LABELS}")
print(f"id2label: \n{id2label}")
print(f"label2id: \n{label2id}")

List of tag values: 
['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']
Number of NER Tags: 
9
id2label: 
{0: 'O', 1: 'art', 2: 'building', 3: 'event', 4: 'location', 5: 'organization', 6: 'other', 7: 'person', 8: 'product'}
label2id: 
{'O': 0, 'art': 1, 'building': 2, 'event': 3, 'location': 4, 'organization': 5, 'other': 6, 'person': 7, 'product': 8}


#### Basic Values/Constants

In [10]:
MODEL_CKPT = "bert-base-cased"

MODEL_NAME = f"{MODEL_CKPT}-finetuned-ner-DFKI-SLT_few-NERd"
NUM_OF_EPOCHS = 2

BATCH_SIZE = 16
STRATEGY = "epoch"

REPORTS_TO = "tensorboard"
WEIGHT_DECAY = 0.01

LR = 2e-5
DEVICE = torch.device("cpu")

STEPS = 1200

#### Function to Tokenize & Align Inputs

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

def tokenize_and_align_labels(samples):
    tokenized_inputs = tokenizer(samples["tokens"],
                                      truncation=True,
                                      is_split_into_words=True)

    labels = []

    for idx, label in enumerate(samples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        prev_word_idx = None
        label_ids = []
        for word_idx in word_ids: # set special tokens to -100
            if word_idx is None or word_idx == prev_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            prev_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

#### Apply Above Function to Tokenize Dataset

In [12]:
encoded_ds = ds.map(tokenize_and_align_labels,
                       batched=True,
                       remove_columns=
                        [
                            'ner_tags',
                            'tokens'
                        ]
                    )

Map:   0%|          | 0/361398 [00:00<?, ? examples/s]

Map:   0%|          | 0/95714 [00:00<?, ? examples/s]

Map:   0%|          | 0/56999 [00:00<?, ? examples/s]

#### Define Data Collator

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

#### Load Custom Model

In [14]:
model = (AutoModelForTokenClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=NUM_OF_LABELS,
    id2label=id2label,
    label2id=label2id
    ).to(DEVICE))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

#### Create Compute Metrics Function

In [15]:
label_list = tag_values

seqeval = evaluate.load("seqeval")

labels = [label_list[i] for i in example[f'ner_tags']]

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = np.argmax(predictions,
                            axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions,
                              references=true_labels)

    return results

#### Define TrainingArguments

In [16]:
args = TrainingArguments(
    MODEL_NAME,
    log_level="error",
    logging_first_step=True,
    learning_rate=LR,
    num_train_epochs=NUM_OF_EPOCHS,
    auto_find_batch_size=True,
    evaluation_strategy=STRATEGY,
    report_to=REPORTS_TO,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    disable_tqdm=False,
    logging_steps=STEPS,
    weight_decay=WEIGHT_DECAY,
    save_strategy=STRATEGY,
    hub_private_repo=True,
    push_to_hub=True
)

#### Define Trainer

In [17]:
trainer = Trainer(model,
                  args=args,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer,
                  train_dataset=encoded_ds["train"],
                  eval_dataset=encoded_ds["test"]
                  )

Cloning https://huggingface.co/DunnBC22/bert-base-cased-finetuned-ner-DFKI-SLT_few-NERd into local empty directory.


#### Train Model

In [18]:
train_results = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Erson,Ocation,Rganization,Roduct,Rt,Ther,Uilding,Vent,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
0,0.179600,0.142667,"{'precision': 0.8740795762821341, 'recall': 0.9272010962658445, 'f1': 0.8998570336137248, 'number': 29190}","{'precision': 0.8576076215009827, 'recall': 0.8071585327620441, 'f1': 0.8316186723086282, 'number': 95690}","{'precision': 0.7699032109387003, 'recall': 0.7688047497046776, 'f1': 0.7693535882339395, 'number': 65183}","{'precision': 0.6710836277974087, 'recall': 0.75, 'f1': 0.7083506009117282, 'number': 9116}","{'precision': 0.834716121685375, 'recall': 0.8153503893214683, 'f1': 0.8249196141479099, 'number': 6293}","{'precision': 0.6742843680056544, 'recall': 0.8195289569761615, 'f1': 0.7398455423789058, 'number': 13969}","{'precision': 0.4812014282713716, 'recall': 0.3950681151922745, 'f1': 0.4339015151515152, 'number': 5799}","{'precision': 0.5997923695821438, 'recall': 0.32526389866291344, 'f1': 0.4217922978645739, 'number': 7105}",0.799997,0.785182,0.792520,0.948302
1,0.154200,0.131176,"{'precision': 0.8860048426150121, 'recall': 0.9401849948612538, 'f1': 0.912291199202194, 'number': 29190}","{'precision': 0.8686381704207632, 'recall': 0.8152889539136796, 'f1': 0.841118472477534, 'number': 95690}","{'precision': 0.7919078915181266, 'recall': 0.7449641777764141, 'f1': 0.7677190874452579, 'number': 65183}","{'precision': 0.7065968977761166, 'recall': 0.8295304958315051, 'f1': 0.7631446160056513, 'number': 9116}","{'precision': 0.8407258064516129, 'recall': 0.8614333386302241, 'f1': 0.8509536143159878, 'number': 6293}","{'precision': 0.7303024586555996, 'recall': 0.8314124132006586, 'f1': 0.7775843599357258, 'number': 13969}","{'precision': 0.5162234691388143, 'recall': 0.3648904983617865, 'f1': 0.4275611234592847, 'number': 5799}","{'precision': 0.605920892987139, 'recall': 0.35144264602392683, 'f1': 0.44486014608943525, 'number': 7105}",0.820252,0.788603,0.804117,0.949809


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: building seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: location seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: organization seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: event seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: product seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequ

#### Push Model To Hub

In [19]:
kwargs = {
    "tasks": "token-classification",
    "tags": ['token-classification'],
}

if args.push_to_hub:
    trainer.push_to_hub('All DUNN!!!', **kwargs)
else:
    trainer.create_model_card(**kwargs)

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/411M [00:00<?, ?B/s]

Upload file runs/Jul05_17-28-12_5a43ec1e4b0f/events.out.tfevents.1688578106.5a43ec1e4b0f.3765.0:   0%|        …

To https://huggingface.co/DunnBC22/bert-base-cased-finetuned-ner-DFKI-SLT_few-NERd
   36a751a..2014d20  main -> main

   36a751a..2014d20  main -> main

To https://huggingface.co/DunnBC22/bert-base-cased-finetuned-ner-DFKI-SLT_few-NERd
   2014d20..f7d2376  main -> main

   2014d20..f7d2376  main -> main



#### Save & Log Model

In [20]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

To https://huggingface.co/DunnBC22/bert-base-cased-finetuned-ner-DFKI-SLT_few-NERd
   f7d2376..ccebf00  main -> main

   f7d2376..ccebf00  main -> main



***** train metrics *****
  epoch                    =        2.0
  total_flos               = 21178098GF
  train_loss               =     0.1765
  train_runtime            = 3:30:48.42
  train_samples_per_second =     57.145
  train_steps_per_second   =      1.786


### Error Analysis

#### Define Method to Apply to Validation Dataset (& Then Apply it)

In [55]:
def forward_pass_with_label(batch):
    # Convert dict of lists to list of dicts suitable for data collator
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # Pad inputs and labels and put all tensors on device
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    labels = batch["labels"].to(DEVICE)
    with torch.no_grad():
        # Pass data through model
        output = trainer.model(input_ids, attention_mask)
        # Logit.size: [batch_size, sequence_length, classes]
        predicted_label = torch.argmax(output.logits, axis=-1).cpu().numpy().to(DEVICE)

    # Calculate loss per token after flattening batch dimension with view
    loss = cross_entropy(output.logits.view(-1, 9), labels.view(-1), reduction="none")
    # Unflatten batch dimension and convert to numpy array
    loss = loss.view(len(input_ids), -1).cpu().numpy().to(DEVICE)

    return {"loss":loss, "predicted_label": predicted_label}

#### Apply Above Function to Entire Evaluation Dataset

In [60]:
eval_set = encoded_ds['eval']

eval_set = eval_set.remove_columns(['id', 'fine_ner_tags'])

eval_set = eval_set.map(forward_pass_with_label,
                        batched=True,
                        batch_size=32)

eval_df = eval_set.to_pandas()

Map:   0%|          | 0/56999 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 5>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:580 in wrapper                 │
│                                                                                                  │
│    577 │   │   else:                                                                             │
│    578 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    579 │   │   # apply actual function                                                           │
│ ❱  580 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    581 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    582 │   │   for dataset in datasets:                                                          │
│    583 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:545 in wrapper                 │
│                                                                                                  │
│    542 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    543 │   │   }                                                                                 │
│    544 │   │   # apply actual function                                                           │
│ ❱  545 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    546 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    547 │   │   # re-apply format to the output                                                   │
│    548 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:3087 in map                    │
│                                                                                                  │
│   3084 │   │   │   │   │   leave=False,                                                          │
│   3085 │   │   │   │   │   desc=desc or "Map",                                                   │
│   3086 │   │   │   │   ) as pbar:                                                                │
│ ❱ 3087 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwargs):     │
│   3088 │   │   │   │   │   │   if done:                                                          │
│   3089 │   │   │   │   │   │   │   shards_done += 1                                              │
│   3090 │   │   │   │   │   │   │   logger.debug(f"Finished processing shard number {rank} of {n  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:3463 in _map_single            │
│                                                                                                  │
│   3460 │   │   │   │   │   │   │   range(*(slice(i, i + batch_size).indices(shard.num_rows)))    │
│   3461 │   │   │   │   │   │   )  # Something simpler?                                           │
│   3462 │   │   │   │   │   │   try:                                                              │
│ ❱ 3463 │   │   │   │   │   │   │   batch = apply_function_on_filtered_inputs(                    │
│   3464 │   │   │   │   │   │   │   │   batch,              

#### Clean Up Padding Tokens

In [58]:
id2label[-100] = "IGN"
eval_df["input_tokens"] = eval_df["input_ids"].apply(
    lambda x: tokenizer.convert_ids_to_tokens(x))
eval_df["predicted_label"] = eval_df["predicted_label"].apply(
    lambda x: [id2label[i] for i in x])
eval_df["labels"] = eval_df["labels"].apply(
    lambda x: [id2label[i] for i in x])
eval_df['loss'] = eval_df.apply(
    lambda x: x['loss'][:len(x['input_ids'])], axis=1)
eval_df['predicted_label'] = eval_df.apply(
    lambda x: x['predicted_label'][:len(x['input_ids'])], axis=1)
eval_df.head(1)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'eval_df' is not defined

#### Unwrap Each Token Within Sample Separately

In [24]:
eval_df_tokens = eval_df.apply(pd.Series.explode)
eval_df_tokens = eval_df_tokens.query("labels != 'IGN'")
eval_df_tokens["loss"] = eval_df_tokens["loss"].astype(float).round(2)
eval_df_tokens.head(7)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'eval_df' is not defined

#### See Which Tokens Have Accumulated Most Loss in Evaluation Dataset

In [ ]:
(
    eval_df_tokens.groupby("input_tokens")[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1) # Get rid of multi-level columns
    .sort_values(by="sum", ascending=False)
    .reset_index()
    .round(3)
    .head(10)
    .T
)

#### See Which Label IDs Have Most Loss in Evaluation Dataset

In [ ]:
(
    eval_df_tokens.groupby("labels")[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1) # Get rid of multi-level columns
    .sort_values(by="mean", ascending=False)
    .reset_index()
    .round(3)
    .fillna(0)
    .T
)

#### Create Function to Display Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

#### Display Confusion Matrix

In [ ]:
eval_token_list = list(set(eval_df_tokens['labels']))

plot_confusion_matrix(eval_df_tokens["labels"], eval_df_tokens["predicted_label"],
                      eval_token_list)

### Notes & Other Takeaways From This Project

****
-

****

### Citations

- Dataset

    > @inproceedings{ding-etal-2021-nerd,
    title = "Few-{NERD}: A Few-shot Named Entity Recognition Dataset",
    author = "Ding, Ning and Xu, Guangwei and Chen, Yulin and Wang, Xiaobin and Han, Xu and Xie, Pengjun and Zheng, Haitao and Liu, Zhiyuan", booktitle = "Proceedings of the 59th Annual Meeting of the Association for Computational Linguistics and the 11th International Joint Conference on Natural Language Processing (Volume 1: Long Papers)", month = aug, year = "2021", address = "Online", publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.acl-long.248", doi = "10.18653/v1/2021.acl-long.248", pages = "3198--3213",}

- Model Checkpoint

    > @article{DBLP:journals/corr/abs-1810-04805, author = {Jacob Devlin and Ming{-}Wei Chang and Kenton Lee and Kristina Toutanova}, title = {{BERT:} Pre-training of Deep Bidirectional Transformers for Language Understanding}, journal = {CoRR}, volume = {abs/1810.04805}, year = {2018}, url = {http://arxiv.org/abs/1810.04805}, archivePrefix = {arXiv}, eprint = {1810.04805}, timestamp = {Tue, 30 Oct 2018 20:39:56 +0100}, biburl = {https://dblp.org/rec/journals/corr/abs-1810-04805.bib}, bibsource = {dblp computer science bibliography, https://dblp.org}}

- Metric (SeqEval)

    > @inproceedings{ramshaw-marcus-1995-text, title = "Text Chunking using Transformation-Based Learning", author = "Ramshaw, Lance and Marcus, Mitch", booktitle = "Third Workshop on Very Large Corpora", year = "1995", url = "https://www.aclweb.org/anthology/W95-0107",}
    
    > @misc{seqeval, title={{seqeval}: A Python framework for sequence labeling evaluation}, url={https://github.com/chakki-works/seqeval}, note={Software available from https://github.com/chakki-works/seqeval}, author={Hiroki Nakayama}, year={2018},}